# Convert the annotations to YOLO format to be compatible with the model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
train_images_folder = '/content/drive/My Drive/Tree in Forest/Supermarket shelves/images/train/'
val_images_folder = '/content/drive/My Drive/Tree in Forest/Supermarket shelves/images/val/'

train_annotations_folder = '/content/drive/My Drive/Tree in Forest/Supermarket shelves/annotations/train/'
val_annotations_folder = '/content/drive/My Drive/Tree in Forest/Supermarket shelves/annotations/val/'

train_labels_folder = '/content/drive/My Drive/Tree in Forest/Supermarket shelves/labels/train/'
val_labels_folder = '/content/drive/My Drive/Tree in Forest/Supermarket shelves/labels/val/'

In [4]:
import os
import json

In [8]:
def get_class_nbr(class_title):
  if class_title == 'Product':
    return 0
  elif class_title == 'Price':
    return 1
  else:
    raise Exception('Unknown class title')

def convert_to_yolo(file_data, file_name, write_to_folder):
  objects = file_data['objects']
  img_width = file_data['size']['width']
  img_height = file_data['size']['height']

  txt_filename = os.path.join(write_to_folder, file_name.split('.')[0] + '.txt')
  with open(txt_filename, 'w') as tf:
    for obj in objects:
      # get class title
      class_title = obj['classTitle']
      try:
        class_nbr = get_class_nbr(class_title)
      except Exception as e:
        print(f'{e}: {class_title}')
        continue   # skip the current object with unknown class title

      # get top-left and bottom-right coordinates
      top_left = tuple(obj['points']['exterior'][0])
      bottom_right = tuple(obj['points']['exterior'][1])

      # get x_center and y_center
      x_center = (top_left[0] + bottom_right[0]) / 2
      y_center = (top_left[1] + bottom_right[1]) / 2

      # remove if center goes past image dimensions
      if (x_center < 0) or (x_center > img_width) or (y_center < 0) or (y_center > img_height):
        continue

      # get width and height
      width = abs(bottom_right[0] - top_left[0])
      height = abs(bottom_right[1] - top_left[1])

      # normalize
      x_center /= img_width
      y_center /= img_height
      width /= img_width
      height /= img_height

      # write to file
      tf.write(f'{class_nbr} {x_center} {y_center} {width} {height}\n')

In [12]:
for file in os.listdir(train_annotations_folder):
  with open(os.path.join(train_annotations_folder, file), 'r') as f:
    data = json.load(f)
    convert_to_yolo(data, file, train_labels_folder)

for file in os.listdir(val_annotations_folder):
  with open(os.path.join(val_annotations_folder, file), 'r') as f:
    data = json.load(f)
    convert_to_yolo(data, file, val_labels_folder)